In [1]:
%pip install --upgrade yfinance

Note: you may need to restart the kernel to use updated packages.


In [7]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define the tickers, including the S&P 500
tickers = [ "AAPL","AMZN","GOOG","NKE","RY.TO",]  # RY.TO is for Royal Bank of Canada, ^GSPC is the S&P 500

# Download historical data (e.g., for the past year)
data = yf.download(tickers, period="1y")

print(data.head(5))



[*********************100%***********************]  5 of 5 completed

Price            Close                                                  \
Ticker            AAPL        AMZN        GOOG         NKE       RY.TO   
Date                                                                     
2024-02-20  180.706726  167.080002  141.690872  101.557899  127.835495   
2024-02-21  181.463165  168.589996  143.325012  103.494675  127.218254   
2024-02-22  183.503525  174.580002  144.799728  103.307877  127.594383   
2024-02-23  181.662231  174.990005  144.769821  103.848602  128.481659   
2024-02-26  180.308624  174.729996  138.253250  102.776993  127.613686   

Price             High                                                  ...  \
Ticker            AAPL        AMZN        GOOG         NKE       RY.TO  ...   
Date                                                                    ...   
2024-02-20  181.572633  168.710007  142.815828  102.088786  128.597390  ...   
2024-02-21  182.030479  170.229996  143.464510  103.740459  128.066944  ...   
2024-02-22  

In [ ]:
# --- Calculate Daily Returns ---
returns = data['Adj Close'].pct_change()

# --- Calculate Correlations ---

In [ ]:
correlation_matrix = returns.corr()

# --- Calculate Rolling Volatility (e.g., 30-day) ---
volatility = returns.rolling(window=30).std()

# --- Calculate Cumulative Returns ---
cumulative_returns = (1 + returns).cumprod() - 1

# --- Risk-Free Rate (Example: 10-year US Treasury yield - you'll want to update this) ---
risk_free_rate = yf.download("^TNX", period="1y")['Adj Close'].iloc[-1] / 100 # Annualized risk-free rate.  Divide by 100 to convert percentage to decimal.
# Important: ^TNX is for the 10-year US Treasury.  You might want to use a more appropriate risk-free rate for your analysis (e.g., a Canadian government bond yield if analyzing Canadian stocks).

# --- Sharpe Ratio Calculation (Annualized) ---
def calculate_sharpe_ratio(returns, risk_free_rate, annualization_factor=252): # 252 trading days in a year
    excess_returns = returns - risk_free_rate/annualization_factor # Daily excess return
    annualized_excess_returns = excess_returns.mean() * annualization_factor # Annualize excess return
    annualized_volatility = volatility.mean() * np.sqrt(annualization_factor) # Annualize volatility
    sharpe_ratio = annualized_excess_returns / annualized_volatility
    return sharpe_ratio


sharpe_ratios = calculate_sharpe_ratio(returns, risk_free_rate)


# --- Export to Excel (separate sheets) ---
with pd.ExcelWriter("stock_analysis.xlsx") as writer:
    data['Adj Close'].to_excel(writer, sheet_name="Adj Close Prices")  # Adjusted Close Prices
    returns.to_excel(writer, sheet_name="Daily Returns")
    correlation_matrix.to_excel(writer, sheet_name="Correlation Matrix")
    volatility.to_excel(writer, sheet_name="Volatility")
    cumulative_returns.to_excel(writer, sheet_name="Cumulative Returns")
    pd.DataFrame(sharpe_ratios, columns=['Sharpe Ratio']).to_excel(writer, sheet_name="Sharpe Ratios") # Sharpe Ratios


print("Data and analysis saved to stock_analysis.xlsx")

# --- Example of Accessing Data (for us) ---
#print("\nCorrelation Matrix:")
#print(correlation_matrix)

#print("\nVolatility:")
#print(volatility)

#print("\nCumulative Returns:")
#print(cumulative_returns)

# --- Further Analysis (Examples for us) ---
# ... (as before)

# Example: Analyzing correlations with the S&P 500
sp500_corr = correlation_matrix["^GSPC"]
print("\nCorrelation with S&P 500:")
print(sp500_corr)

print("\nSharpe Ratios:")
print(sharpe_ratios)